<a href="https://colab.research.google.com/github/VicentePina7210/DataMiningCleaningExercise/blob/main/AssociationRuleMiningExercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Association Rule Mining

In [3]:
import pandas as pd
import numpy as np
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("heeraldedhia/groceries-dataset")

groceries_df = pd.read_csv(os.path.join(path, "Groceries_dataset.csv"))

100%|██████████| 257k/257k [00:00<00:00, 51.0MB/s]

Extracting files...


In [4]:
# 1. Format dataset so each row contains a list of items corresponding to a member number
purchased_group = groceries_df.groupby(["Member_number", "Date"])["itemDescription"].apply(list)
transaction_ids = [str(mid) + ":" + date for mid, date in purchased_group.index]
items_lists = list(purchased_group)
items_df = pd.DataFrame(list(zip(transaction_ids, items_lists)), columns=["transaction", "items"])
items_df

,transaction,items
0,1000:15-03-2015,"[sausage, whole milk, semi-finished bread, yog..."
1,1000:24-06-2014,"[whole milk, pastry, salty snack]"
2,1000:24-07-2015,"[canned beer, misc. beverages]"
3,1000:25-11-2015,"[sausage, hygiene articles]"
4,1000:27-05-2015,"[soda, pickled vegetables]"
...,...,...
14958,4999:24-01-2015,"[tropical fruit, berries, other vegetables, yo..."
14959,4999:26-12-2015,"[bottled water, herbs]"
14960,5000:09-03-2014,"[fruit/vegetable juice, onions]"
14961,5000:10-02-2015,"[soda, root vegetables, semi-finished bread]"


In [5]:
# 2. Define support and confidence
def support_count(itemset, items_df):
  count = 0
  for index, row in items_df.iterrows():
    if all(item in row["items"] for item in itemset): count += 1
  return count

def support(itemset, items_df):
  return support_count(itemset, items_df) / items_df.shape[0]

def confidence(rule, items_df):
  return support_count(rule[0] + rule[1], items_df) / support_count(rule[0], items_df)

support(["sausage", "hygiene articles"], items_df)
#confidence([["sausage"], ["hygiene articles"]], items_df)

0.0008688097306689834

In [33]:
# 3. Create a function to generate frequent item sets. What min support threshold works best?
def generate_frequent_sets(itemsets, min_support):
    frequent_itemsets = []  # List to store frequent itemsets
    for item in itemsets:
        item_support = support(item, items_df)  # Use the support function defined earlier
        if item_support >= min_support:  # Check if the itemset meets the min support threshold
            print(item, item_support)  # Print the itemset and its support
            frequent_itemsets.append(item)  # Add the itemset to the list of frequent itemsets
    return itemsets  # Return the list of frequent itemsets


In [34]:
# 4. Create a function to generate a set of confident rules from the itemsets
# Represent "{Milk, Bread} -> {Eggs}" as [["milk", "bread"], ["eggs"]]

def generate_subsets(itemset):
    subsets = []
    # Generate all non-empty subsets by iterating over possible sizes
    for size in range(1, len(itemset) + 1):
        for i in range(len(itemset) - size + 1):
            subsets.append(itemset[i:i+size])
    return subsets

# Function to generate confident rules
def generate_confident_rules(itemsets, min_confidence, items_df):
    confident_rules = []  # List to store confident rules

    # For each itemset, generate all possible non-empty subsets (antecedents)
    for itemset in itemsets:
        # Generate all possible non-empty subsets (antecedents)
        antecedents = generate_subsets(itemset)

        for antecedent in antecedents:
            consequent = list(set(itemset) - set(antecedent))  # The remaining items are the consequent
            rule = [antecedent, consequent]

            # Calculate confidence for the rule
            rule_confidence = confidence(rule, items_df)

            if rule_confidence >= min_confidence:
                confident_rules.append(rule)  # Add the rule if it meets the confidence threshold

    return confident_rules

In [35]:
# 5.a. Sort the generated rules by confidence and examine the top rules.

min_confidence = 0.7  # Set a minimum confidence threshold
confident_rules = generate_confident_rules(itemsets, min_confidence, items_df)

for rule in confident_rules:
    print(f"Rule: {rule[0]} -> {rule[1]}")

# 5.b. What interesting or unexpected patterns do you see?


KeyboardInterrupt: 